# Finite differences in 1D

We approximate the boundary value problem consisting of the following PDE:
$$
-u''(x) + u(x) = f(x), \qquad  0 < x < 1.
$$
(This is actually an ODE since the unknown is a function of a single real variable $x$.) This equation is supplemented with  Dirichlet boundary conditions 
$$
u(0) = u(1) = 0
$$
at both ends. 

The purpose of this notebooks is to quickly illustrate the standard finite difference method for solving this, taking the opportunity to also ensure that we are all on the same page regarding the python  prerequisites. Let's begin by importing the `numpy` module.

In [ ]:
import numpy as np

## Approximating the derivative 

It is very easy to understand and implement finite differences. The finite difference approximation of $u(x)$ is a finite sequence $u_i$. Each $u_i$ approximates $u(x_i)$ at a point $x_i$ in the interval $[0, 1]$ where we need the solution. The sequence of points $x_i$ may be thought of a grid or mesh of the domain $[0, 1]$.

To approximate the ODE, we need to approximate derivatives using the grid. The **forward difference** approximation of $d u / dx$ is 
$$
[D_+u]_i = \frac{ u(x_{i+1}) - u(x_i) }{ x_{i+1} - x_i}, \qquad
u'(x_i) \approx [D_+u]_i.
$$
The **backward difference** approximation is 
$$
[D_-u]_i = \frac{ u(x_{i}) - u(x_{i-1}) }{ x_i - x_{i-1}}, \qquad
u'(x_i) \approx [D_+u]_i.
$$
Note that by Taylor's theorem, when $u$ is smooth, both approximate the derivative as $x_{i\pm 1} \to x_i$:
$$
\frac{d u }{d x}(x_i) \approx [D_+ u]_i, \qquad \frac{d u }{d x}(x_i) \approx [D_- u]_i. 
$$
In fact, from calculus tools, you immediately see that both the approximations are $O(h)$ where $h$ is the spacing between the points on a uniform grid.

## The finite difference system

For approximating the ODE $-u'' +u = f$, we need to approximate the second derivative, not the first. Obviously there are many ways to combine the above two differences to get an approximation to $u''$, 
$$
\frac{d }{dx} \frac{d u}{d x} (x_i) \approx [D_\pm D_\pm u]_i.
$$
Depending on the choice in $\pm$, we have four possible approximations. 

However, some are $O(h^2)$ accurate, while others are only $O(h)$-accurate. (**Exercise:** Which? Why?)

The **Central Difference Formula** for approximating the second derivative is 
$$
\frac{d }{dx} \frac{d u}{d x} (x_i) \approx [D_+D_- u]_i \equiv [D_-D_+ u]_i,
$$
which can be alternately written on a uniform grid of mesh size $h$ as
$$
\begin{aligned}
u''(x_i) 
& \approx {u_{i+1}  - 2 u_i + u_{i-1} \over h^2}.
\end{aligned}
$$

## Implementing the finite difference system


The **matrix** of the *central second finite difference* operator on a grid of *just* 5 equally spaced points, two of which have zero boundary conditions, can be "made by hand" as a `numpy` array, as follows (save for a  factor of $-1/h^2$).

In [ ]:
A = np.array([[ 2, -1,  0], 
              [-1,  2, -1], 
              [ 0, -1,  2]])
A

For large number of grid points, we need an automatic way to make this matrix. Numpy provides many ways to create matrices quickly. For example, the `diag` command generates matrices with input entries in the diagonal, superdiagonals, or subdiagonals.

In [ ]:
N = 10
2 * np.diag(np.ones(N))

In [ ]:
A = 2 * np.diag(np.ones(N)) +    \
    np.diag(-np.ones(N-1), -1) + \
    np.diag(-np.ones(N-1), 1) 
A

This `A` when multiplied by a vector of values of $u/h^2$ gives approximate values of $-u''$. Using this we can approximate the left hand side of the PDE $-u'' + u = f$, as done next.

## Solving the difference equation system


Adding values of $u$ to the discretization of $-u''$ we obtain the left hand side of the finite difference system. The right hand side just consists of a vector values of $f$ at the grid points.

We solve the finite difference equations using the built-in inverse routine in numpy's `linalg` submodule. (Note the definition of a python function and how we use `@` for numpy's matrix multiplication.)

In [ ]:
def solve(f, h):
    size = len(f)
    A = (1/h**2) * (2 * np.diag(np.ones(size))    + \
                    np.diag(-np.ones(size-1), -1) + 
                    np.diag(-np.ones(size-1), 1)) + np.eye(size)
    return np.linalg.inv(A) @ f

Now you have an approximate solution to the boundary value problem in the returned vector. Is it any good? 

## Check for correctness

**Verification** is a key step while designing and implementing  numerical methods.

We will verify that the `solve` function is solving as expected by the **method of manufactured solutions**, which is just a fancy name for  the following simple idea: pick your favorite smooth function $u$ satisfying the boundary conditions, then generate the right hand side $f$ that would give your $u$ as the exact solution by applying the differential operator to $u$. 

In the check below, I will use
$$
u = \sin(x)
$$
on an interval $(0, 3\pi)$ so that the zero boundary conditions hold. Then put $f = -u'' + u = 2\sin(x)$. Let's provide this $f$ to `solve` and see whether it outputs an approximation to $u = \sin(x)$.

An equally spaced grid in any interval is easily made using the `linspace` facility.

In [ ]:
np.linspace(0, 3*np.pi, num=10)

Let's "manufacture"  the data $f$ for the manufactured solution $u=\sin(x)$.

In [ ]:
N = 30    # the system becomes more expensive to solve for large N
h = 3*np.pi / N
x = np.linspace(0, 3*np.pi, num=N)
f = 2 * np.sin(x)

One point to note about the boundary points: when we solve, we make sure not to give the end point values in `f` (as the solution there is already determined by the boundary conditions).  Restricting $f$ outside of these points is done by **slicing** in numpy (which you should definitely learn if you don't know already) as in `f[1:-1]`.

In [ ]:
u = solve(f[1:-1], h)

For visualizing the solution, we use `matplotlib` module.

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.plot(u, '.-')

How can you make sure the end points with the known zero solution values  are also included in the final plot? This is again done using slicing. 

In [ ]:
uu = np.zeros(N)
uu[1:-1] = u

Let's conclude by comparing the exact solution with the numerical solution by plotting both in the same scene. 

In [ ]:
plt.plot(uu, '.', label='computed solution')
plt.plot(np.sin(x), ':', label='exact solution', alpha=0.5)
plt.grid(True)
plt.legend();

Clearly, the `solve` function captured something close to the exact solution.

Going back through this notebook, you can easily change the mesh parameter to make $h$ smaller. Then repeating the above steps, you will see that the discrete solution points gets closer to exact solution curve. 

We will proceed to study finite elements. Then, unlike the above,  we will think of the discrete solution as a function (and not as a set of discrete values as above). The function will be from a finite-dimensional space (called a finite element space).